# Create Individual Deforestation Files From Hansen Dataset

The purpose of this notebook is two fold:
1. Extract pixels:Takes the 'lossyear' image from the Hansen et al. (2013) dataset and creates a tiff file deforestation_year for each year desired. Take the 'treecover2000' and filter for pixels greater than 10%. 
2. 'lossyear' and 'treecover2000' also need to be cropped so that pixels are only within the boundary of the active property of that year. 

# Import  libraries 

In [1]:
import os
import sys
from pathlib import Path
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.transform import from_origin
import geopandas as gpd
import pandas as pd
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt


# Import Constants

In [2]:

# Get the current working directory
current_dir = os.path.abspath('')

# Search for the 'constants.py' file starting from the current directory and moving up the hierarchy
project_root = current_dir
while not os.path.isfile(os.path.join(project_root, 'constants.py')):
    project_root = os.path.dirname(project_root)

# Add the project root to the Python path
sys.path.append(project_root)


In [4]:
from constants import HANSEN_LOSSYEAR_FILEPATHS, HANSEN_TREECOVER_FILEPATH, STUDY_BOUNDARY_PATH, OUTPUT_PATH

In [5]:
study_boundary_gdf = gpd.read_file(STUDY_BOUNDARY_PATH)

'''rasterio.open() expects a single file path as a string, 
 pass the file path string directly. Since HANSEN_LOSSYEAR_FILEPATHS is a list, 
 you can access the file path string by indexing the list with [0].
 '''
# Reproject the study boundary to match the Hansen raster CRS
with rasterio.open(HANSEN_LOSSYEAR_FILEPATHS[0]) as src:
    hansen_crs = src.crs

# Reproject the study boundary GeoDataFrame to match the Hansen raster CRS    
study_boundary_gdf_reprojected = study_boundary_gdf.to_crs(hansen_crs)
# Get the geometry column from the reprojected GeoDataFrame
study_boundary_geom_reprojected = study_boundary_gdf_reprojected.geometry

In [6]:
with rasterio.open(HANSEN_LOSSYEAR_FILEPATHS[0]) as hansen:
    hansen_array = hansen.read(1)
    hansen_crs = hansen.crs
    out_transform = hansen.transform
    out_meta = hansen.meta

# Extract Pixel Function

In [13]:
'''This function accepts raster data, a start year, and an end year as arguments. 
The function iterates through the years within the given range and 
creates a binary mask for each year where the raster values match the year. 
It then stores these binary masks in a dictionary with the corresponding year 
as the key.'''



# Extract pixels corresponding to each year (2001-2021)
# This will return binary but if we want to keep the encoded value of the 
# pixelReturns pixels encoded with value of 1 and zeros as NaN.
#if `year_pixels[year_pixels == 0] = np.nan` is removed then will return 
# Unique values for year 2011: [0 1]. 

def extract_pixels_by_year_binary(raster_data, start_year, end_year):
    year_data = {}
    for year in range(start_year, end_year + 1):
        year_pixels = (raster_data == year).astype(int) 
        year_data[year] = year_pixels

        # Print unique values for each year
        unique_values = np.unique(year_pixels)
        print(f"Unique values for year {year + 2000}: {unique_values}") # Add 2000 to the year to get the correct year values
    return year_data


'''While iterating through the years, the function also prints unique values for 
each year's binary mask. These unique values should be either 0 or 1, 
where 1 represents the pixels that have a deforestation event 
for that specific year, and 0 represents the pixels that do not.'''





"While iterating through the years, the function also prints unique values for \neach year's binary mask. These unique values should be either 0 or 1, \nwhere 1 represents the pixels that have a deforestation event \nfor that specific year, and 0 represents the pixels that do not."

# Extract Tree Cover >= 10%, Convert to Binary

To extract pixels with 10% and above tree cover and write the resulting raster:

1. Read the tree cover raster data.
2. Create a binary mask for pixels with tree cover equal to or greater than 10%.
3. Write the binary mask to a new raster file.

In [7]:
# Step 1: Read the tree cover raster data
with rasterio.open(HANSEN_TREECOVER_FILEPATH[0]) as src:
    tree_cover_array = src.read(1)
    tree_cover_transform = src.transform
    tree_cover_meta = src.meta

# Step 2: Create a mask for pixels with tree cover equal to or greater than 10%
'''np.where() to create a new array, masked_array,
that contains the original tree cover values 
where the condition (tree cover >= 10%) is met and 0 for the other pixels. '''
tree_cover_threshold = 10
masked_array = np.where(tree_cover_array >= tree_cover_threshold, tree_cover_array, 0)

# Step 3: Write the masked array to a new raster file
output_dir = os.path.join(OUTPUT_PATH[0], 'tree_cover_10_percent_and_above_00')
os.makedirs(output_dir, exist_ok=True)

output_filepath = os.path.join(output_dir, 'tree_cover_10_percent_and_above_00.tif')
with rasterio.open(output_filepath, 'w', **tree_cover_meta) as dst:
    dst.write(masked_array, 1)
    dst.transform = tree_cover_transform

In [9]:
# Step 1: Read the existing tree cover raster
with rasterio.open(output_filepath) as src:
    tree_cover_10_and_above = src.read(1)

# Step 2: Reclassify the tree cover data to have a value of 1 for tree cover >= 10% and 0 for tree cover < 10%
binary_tree_cover_array = np.where(tree_cover_10_and_above > 0, 1, 0)

# Step 3: Write the binary array to a new raster file
output_filepath_treecover2010 = os.path.join(output_dir, 'treecover2010.tif')
with rasterio.open(output_filepath_treecover2010, 'w', **tree_cover_meta) as dst:
    dst.write(binary_tree_cover_array, 1)
    dst.transform = tree_cover_transform

In [23]:
output_dir = os.path.join(OUTPUT_PATH[0], 'deforestation-cumulative_0110')
os.makedirs(output_dir, exist_ok=True)
# Step 1: Read the existing treecover2000 raster
with rasterio.open(os.path.join(OUTPUT_PATH[0], 'tree_cover_10_percent_and_above_00','tree_cover_10_percent_and_above_00.tif')) as src:
    treecover2000 = src.read(1)

# Step 2: Extract binary masks for each year in the lossyear raster
pixels_by_year = extract_pixels_by_year_binary(hansen_array, 1, 10)

# Step 3: Create a binary deforestation raster for years 1 to 10 (2001 to 2010)
deforestation_1_to_10_binary = np.zeros_like(hansen_array, dtype=int)

for year in range(1, 11):  # We use 11 because range's upper bound is exclusive
    deforestation_1_to_10_binary = np.logical_or(deforestation_1_to_10_binary, pixels_by_year[year]).astype(int)

# Step 4: Apply the deforestation mask to the treecover2000 raster
# Pixels without deforestation = 1, pixels with deforestation = 0
treecover2010 = np.where(deforestation_1_to_10_binary == 1, 0, 1)

# Step 5: Write the treecover2010 array to a new raster file
output_filepath_treecover2010 = os.path.join(output_dir, 'treecover2010.tif')
with rasterio.open(output_filepath_treecover2010, 'w', **out_meta) as dst:
    dst.write(treecover2010, 1)
    dst.transform = out_transform



Unique values for year 2001: [0 1]
Unique values for year 2002: [0 1]
Unique values for year 2003: [0 1]
Unique values for year 2004: [0 1]
Unique values for year 2005: [0 1]
Unique values for year 2006: [0 1]
Unique values for year 2007: [0 1]
Unique values for year 2008: [0 1]
Unique values for year 2009: [0 1]
Unique values for year 2010: [0 1]


# Create Deforestation 11-20

In [25]:
# Extract pixels corresponding to each year (2011-2020)
pixels_by_year = extract_pixels_by_year_binary(hansen_array, 11, 20)

# Create a binary deforestation raster for years 11 to 20 (2011 to 2020)
deforestation_11_to_20_binary = np.zeros_like(hansen_array, dtype=int)

for year in range(11, 21):  # We use 21 because range's upper bound is exclusive
    deforestation_11_to_20_binary += pixels_by_year[year]

# Convert the values to 1 where there was deforestation (values > 0)
deforestation_11_to_20_binary = (deforestation_11_to_20_binary > 0).astype(int)

# Write this to a new raster file
output_filepath_deforestation11_20 = os.path.join(output_dir, 'deforestation11_20.tif')
with rasterio.open(output_filepath_deforestation11_20, 'w', **out_meta) as dst:
    dst.write(deforestation_11_to_20_binary, 1)
    dst.transform = out_transform


Unique values for year 2011: [0 1]
Unique values for year 2012: [0 1]
Unique values for year 2013: [0 1]
Unique values for year 2014: [0 1]
Unique values for year 2015: [0 1]
Unique values for year 2016: [0 1]
Unique values for year 2017: [0 1]
Unique values for year 2018: [0 1]
Unique values for year 2019: [0 1]
Unique values for year 2020: [0 1]
